# Experimental Analysis of Static Community Detection Using JUST

Running both the Louvain and Leiden algorithms multiple times and recording various statistics for each run can provide valuable insights for our analysis.

## Importing Edge List w/ Weights to NetworkX

NetworkX's read_weighted_edgelist function expects a simple text file with lines of the form <node1> <node2> <weight>, without headers. Since our data is in CSV format, you'll need to use Pandas (or another method) to load the CSV and adjust it to become readable.

In [1]:
import pandas as pd

edge_list_df = pd.read_csv('Input/JUST_edge_list_with_similarity.csv')

print(edge_list_df)

            source    target    weight
0        u74717431  t7748381  0.957671
1       u127821914  t3529910  0.426984
2       u174194590  t5762915  0.369877
3       u141847381  t6987845  0.745354
4        u87215499  t4082536  0.946200
...            ...       ...       ...
641279     ci20717       co3 -0.007842
641280     ci20718       co9  0.022677
641281     ci20719       co5 -0.050816
641282     ci20720       co3 -0.178938
641283     ci20721       co4  0.128145

[641284 rows x 3 columns]


In [2]:
negative_weights = edge_list_df[edge_list_df['weight'] < 0]
print(f"Number of edges with negative weights: {len(negative_weights)}")

Number of edges with negative weights: 102041


Since Louvain is not made to consider negative edge weights, we will rescale the weights such that instead of [-1, 1] being the range, it is now [0, 1], where 0 now represents perfect dissimilarity, 0.5 represents orthogonality, and 1 represents perfect similarity.

In [3]:
edge_list_df['weight'] = (edge_list_df['weight'] + 1) / 2

print(edge_list_df)

            source    target    weight
0        u74717431  t7748381  0.978835
1       u127821914  t3529910  0.713492
2       u174194590  t5762915  0.684939
3       u141847381  t6987845  0.872677
4        u87215499  t4082536  0.973100
...            ...       ...       ...
641279     ci20717       co3  0.496079
641280     ci20718       co9  0.511339
641281     ci20719       co5  0.474592
641282     ci20720       co3  0.410531
641283     ci20721       co4  0.564072

[641284 rows x 3 columns]


Before we continue with the creation of a graph, NetworkX specifies that an undirected, weighted graph must not have self-loop, parallel edges (A->B, B->A), or duplicate edges.

In [4]:
duplicate_edges = edge_list_df.duplicated(subset=['source', 'target'], keep=False)
print(f"Number of duplicate edges: {duplicate_edges.sum()}")

self_loops = edge_list_df[edge_list_df['source'] == edge_list_df['target']]
print(f"Number of self-loops: {len(self_loops)}")

print(edge_list_df.isnull().sum())

Number of duplicate edges: 0
Number of self-loops: 0
source    0
target    0
weight    0
dtype: int64


In [5]:
# Find duplicate edges (ignoring the weight column)
duplicate_edges = edge_list_df.duplicated(subset=['source', 'target'], keep=False)

# Filter to get only the duplicate edges
parallel_edges_df = edge_list_df[duplicate_edges]

# Sort to better visualize parallel edges
parallel_edges_sorted = parallel_edges_df.sort_values(by=['source', 'target'])

print(parallel_edges_sorted)

Empty DataFrame
Columns: [source, target, weight]
Index: []


We check for any non-numeric values in the weight column, since this will not be valid when input into a graph object.

In [6]:
# Check for any non-numeric values in the 'weight' column
non_numeric_weights = edge_list_df[pd.to_numeric(edge_list_df['weight'], errors='coerce').isna()]

# Display rows with non-numeric or NaN weights
print(non_numeric_weights)

Empty DataFrame
Columns: [source, target, weight]
Index: []


In [7]:
# Convert the 'weight' column to floating point values
edge_list_df['weight'] = edge_list_df['weight'].astype(float)

# Check the data type of the column to confirm the conversion
print(edge_list_df.dtypes)

source     object
target     object
weight    float64
dtype: object


In [8]:
display(edge_list_df)

,source,target,weight
0,u74717431,t7748381,0.978835
1,u127821914,t3529910,0.713492
2,u174194590,t5762915,0.684939
3,u141847381,t6987845,0.872677
4,u87215499,t4082536,0.973100
...,...,...,...
641279,ci20717,co3,0.496079
641280,ci20718,co9,0.511339
641281,ci20719,co5,0.474592
641282,ci20720,co3,0.410531


## Creating Undirected Weighted NX Graph

We iterate over the edge list DataFrame rows to add edges along with their weights to a new NetworkX graph.


In [9]:
import networkx as nx

def get_graph_info(graph):
    print("Number of nodes:", graph.number_of_nodes())
    print("Number of edges:", graph.number_of_edges())
    
    # Checking the graph type to provide appropriate information
    if isinstance(graph, nx.DiGraph):
        print("Graph is Directed")
    else:
        print("Graph is Undirected")


In [10]:
# Initialize a new graph
G = nx.MultiGraph()

# Add edges and weights
for index, row in edge_list_df.iterrows():
    source = row['source']
    target = row['target']
    weight = row['weight']
    
    # Add the edge with weight
    G.add_edge(source, target, weight=weight)

In [11]:
get_graph_info(G)

Number of nodes: 245760
Number of edges: 641284
Graph is Undirected


## Creating Undirected Weighted iGraph for Leiden

Provided that running Leiden on the MusicMicro dataset was problematic, we decided to isolate the issue and directly use iGraph, as suggested by Leiden's authors.  

We iterate over the edge list DataFrame rows to add edges into a tuple list, along with their weights in a separate list for input into a new iGraph graph.

In [12]:
import igraph as ig

# Assuming edge_list_df is your DataFrame
edges_with_weights = [(row['source'], row['target'], row['weight']) for index, row in edge_list_df.iterrows()]

# Creating the igraph Graph
g = ig.Graph.TupleList(edges_with_weights, edge_attrs={'weight': [w for _, _, w in edges_with_weights]})

# Now, check again if the weights have been correctly assigned
print(g.es['weight'][:5])
print(g.summary())

[0.9788354573755722, 0.7134917524156085, 0.6849385653326384, 0.8726768455516055, 0.9730999967361242]
IGRAPH UNW- 245760 641284 -- 
+ attr: name (v), weight (e)


## Testing Modularity, Run Time and No. of Communities

- Iterates 10 times, running both the Louvain (using CDLib) and Leiden (using iGraph) algorithms on each iteration.
- Records modularity, number of communities, and execution time for each run.
- Calculates the average modularity, average number of communities, and average execution time for both algorithms across all runs.
- Stores all this information in the results dictionary for easy access and analysis.

In [13]:
import time
import numpy as np
from cdlib import algorithms
import leidenalg

# Prepare lists to store results
results = {
    "Louvain": {"modularity": [], "communities": [], "time": []},
    "Leiden": {"modularity": [], "communities": [], "time": []}
}

# Execute each algorithm 10 times
for _ in range(10):
    # Louvain
    start_time = time.time()
    communities_louvain = algorithms.louvain(G)
    elapsed_time = time.time() - start_time
    results["Louvain"]["modularity"].append(communities_louvain.newman_girvan_modularity().score)
    results["Louvain"]["communities"].append(len(communities_louvain.communities))
    results["Louvain"]["time"].append(elapsed_time)
    
    # Leiden
    start_time = time.time()
    partition = leidenalg.find_partition(g, partition_type=leidenalg.ModularityVertexPartition, weights='weight')
    elapsed_time = time.time() - start_time
    results["Leiden"]["modularity"].append(partition.modularity)
    results["Leiden"]["communities"].append(len(partition))
    results["Leiden"]["time"].append(elapsed_time)

# Calculate averages
for method in results:
    results[method]["avg_modularity"] = np.mean(results[method]["modularity"])
    results[method]["avg_time"] = np.mean(results[method]["time"])
    results[method]["avg_communities"] = np.mean(results[method]["communities"])

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'graph_tool', 'bayanpy', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [14]:
# Print or store the results as needed
print(results)

{'Louvain': {'modularity': [0.5656893453610867, 0.64984092144641, 0.6292031128923042, 0.6282723170386361, 0.6543256512558635, 0.5654575762229106, 0.565773856750705, 0.652945267900033, 0.6518107717223783, 0.6527041078458777], 'communities': [53442, 47500, 47478, 47370, 46751, 53331, 53350, 47176, 47187, 45873], 'time': [155.24511098861694, 299.6578559875488, 167.96519112586975, 140.24966883659363, 241.565575838089, 133.41029405593872, 183.64130902290344, 141.2367868423462, 269.01964497566223, 243.0229251384735], 'avg_modularity': 0.6216022928436206, 'avg_time': 197.50143628120424, 'avg_communities': 48945.8}, 'Leiden': {'modularity': [0.5959704298751747, 0.5904229328542269, 0.5911978190177319, 0.5968931287490375, 0.596234834000226, 0.5954904998090195, 0.5911924642374996, 0.5916931290554548, 0.595612127053086, 0.5912204936525011], 'communities': [467, 628, 621, 504, 467, 550, 626, 631, 491, 633], 'time': [17.330739974975586, 17.59577989578247, 20.839152812957764, 15.963630199432373, 16.6

In [15]:
# Convert the results dictionary into a pandas DataFrame
# First, prepare the data in a structured form
data = {
    "Algorithm": [],
    "Run": [],
    "Modularity": [],
    "Communities": [],
    "Time (s)": []
}

# Populate the structured data from the results
for algo in results:
    for run in range(10):  # Assuming 10 runs as previously set
        data["Algorithm"].append(algo)
        data["Run"].append(run + 1)  # Run number (1-10)
        data["Modularity"].append(results[algo]["modularity"][run])
        data["Communities"].append(results[algo]["communities"][run])
        data["Time (s)"].append(results[algo]["time"][run])

# Creating the DataFrame
results_df = pd.DataFrame(data)

# Display the DataFrame for visual inspection
print(results_df)

# Additionally, creating a summary DataFrame for averages
summary_data = {
    "Algorithm": ["Louvain", "Leiden"],
    "Avg. Modularity": [results["Louvain"]["avg_modularity"], results["Leiden"]["avg_modularity"]],
    "Avg. Communities": [results["Louvain"]["avg_communities"], results["Leiden"]["avg_communities"]],
    "Avg. Time (s)": [results["Louvain"]["avg_time"], results["Leiden"]["avg_time"]]
}

summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
print(summary_df)

   Algorithm  Run  Modularity  Communities    Time (s)
0    Louvain    1    0.565689        53442  155.245111
1    Louvain    2    0.649841        47500  299.657856
2    Louvain    3    0.629203        47478  167.965191
3    Louvain    4    0.628272        47370  140.249669
4    Louvain    5    0.654326        46751  241.565576
5    Louvain    6    0.565458        53331  133.410294
6    Louvain    7    0.565774        53350  183.641309
7    Louvain    8    0.652945        47176  141.236787
8    Louvain    9    0.651811        47187  269.019645
9    Louvain   10    0.652704        45873  243.022925
10    Leiden    1    0.595970          467   17.330740
11    Leiden    2    0.590423          628   17.595780
12    Leiden    3    0.591198          621   20.839153
13    Leiden    4    0.596893          504   15.963630
14    Leiden    5    0.596235          467   16.676746
15    Leiden    6    0.595490          550   16.079411
16    Leiden    7    0.591192          626   19.201507
17    Leid

In [16]:
# Save the DataFrame to a CSV file
csv_filename = 'JUST_CD_Experiment_Values.csv'
results_df.to_csv(csv_filename, index=False)